# Benchmark Repeat OPF

This notebook aims to benchmark the repeative optimal power flow.

In [1]:
import warnings
import logging
import ams
import numpy as np
import pandas as pd

ams.config_logger(stream_level=50)

%run ../benchmarks.py
_ = get_tool_versions()

Last run time: 2025-02-02 09:12:44
Python: 3.12.0 | packaged by conda-forge | (main, Oct  3 2023, 08:36:57) [Clang 15.0.7 ]

Tool        Version
----------  -------
ltbams      1.0.3a1
andes       1.9.3  
cvxpy       1.6.0  
gurobipy    12.0.1 
mosek       11.0.5 
piqp        0.4.2  
pandapower  2.14.7 
numba       0.60.0 


In [2]:
cases_lfs = [
    ams.get_case('5bus/pjm5bus_uced.xlsx'),
    ams.get_case('ieee14/ieee14_uced.xlsx'),
    ams.get_case('ieee39/ieee39_uced.xlsx'),
    ams.get_case('npcc/npcc_uced.xlsx'),
    ams.get_case('wecc/wecc_uced.xlsx'),
]

lfs = [ams.load(case, setup=True).EDTSlot.sd.v[:, 0] for case in cases_lfs]

# store loading factor data
lfs_data = pd.DataFrame(data=np.array(lfs).T,
                        columns=['pglib_opf_case5_pjm', 'case14',
                                 'case39', 'npcc', 'wecc'])
lfs_data.to_csv('lfs_data.csv', index=False)

In [3]:
# Suppress warnings
warnings.filterwarnings('ignore')
# show only errors
logger = logging.getLogger('pandapower').setLevel(logging.ERROR)

cases = [
    './cases/case5.m',
    './cases/case14.m',
    './cases/case39.m',
    './cases/npcc.m',
    './cases/wecc.m',
]

case_names = [case.split('/')[-1].split('.')[0] for case in cases]

# Initialize data structures for storing results
n_iters = 10
n_cases = len(cases)

solvers = ['GUROBI', 'MOSEK', 'PIQP', 'pandapower']

time_data = np.zeros((n_iters, n_cases, len(cols_pre) + len(solvers)))
obj_data = np.zeros((n_cases, len(solvers)))

In [4]:
for n_case, (case, lf) in enumerate(zip(cases, lfs)):
    print(f'Case: {case}')
    system = ams.load(case, setup=True, no_output=True, default_config=True)

    for n_iter in range(n_iters):
        pre_time, sol = time_dcopf_with_lf(system,
                                           solvers=solvers,
                                           load_factors=lf,
                                           ignore_dpp=False)
        time_data[n_iter, n_case, :len(pre_time)] = np.array(
            [i for i in pre_time.values()])
        time_data[n_iter, n_case, len(pre_time):] = np.array(
            [sol[solver]['time'] for solver in solvers])

    obj = np.array([sol[solver]['obj'] for solver in solvers])
    obj_data[n_case, :] = obj

Case: ./cases/case5.m
Set parameter Username
Set parameter LicenseID to value 2617183
Academic license - for non-commercial use only - expires 2026-02-02
Case: ./cases/case14.m
Case: ./cases/case39.m
Case: ./cases/npcc.m
Case: ./cases/wecc.m


In [5]:
# NOTE: results structure:
# 1st column: iteration
# 2nd column: cases
# 3rd column: solvers

obj_dcopf = pd.DataFrame(obj_data, columns=solvers, index=case_names)

time_dcopf = pd.DataFrame(columns=cols_pre+solvers, index=case_names)
for case in case_names:
    time_dcopf.loc[case] = time_data[:, case_names.index(case), :].mean(axis=0)

time_dcopf.iloc[:, :] *= 1000        # scale to ms

In [6]:
time_dcopf

,ams_mats,ams_parse,ams_eval,ams_final,ams_postinit,GUROBI,MOSEK,PIQP,pandapower
case5,0.59,1.2,1.79,0.1,0.0,29.57,30.71,19.2,352.98
case14,0.6,1.13,1.71,0.09,0.0,34.6,34.7,20.64,248.71
case39,0.88,1.12,1.87,0.09,0.0,47.38,46.09,23.71,253.26
npcc,1.29,1.13,4.26,0.09,0.0,152.13,142.89,50.7,1205.11
wecc,1.4,1.13,2.81,0.09,0.0,149.62,106.5,50.96,852.12


In [7]:
obj_dcopf.round(2)

,GUROBI,MOSEK,PIQP,pandapower
case5,308818.76,308818.76,308818.76,308818.76
case14,136767.75,136767.75,136767.75,136767.75
case39,643403.27,643403.26,643403.27,643403.27
npcc,16635796.96,16635796.96,16635796.96,16634401.65
wecc,8621568.97,8621568.97,8621568.97,8621568.97


In [8]:
obj_dcopf.to_csv('../results/results_obj_repeat.csv')
time_dcopf.to_csv('../results/results_time_repeat.csv')